In [5]:
import a1chemy
# 申万一级行业分类数据
a1chemy.parse_sw_sectors('/Users/line/workspace/freedom/doc/SwClass.xls', 
                         '/Users/line/workspace/freedom/A1chemy/data/sw_sector.json')
a1chemy.parse_csi_index_xls('/Users/line/workspace/freedom/doc/000906cons.xls',
                        '/Users/line/workspace/freedom/A1chemy/data/csi_800.json')
a1chemy.parse_csi_index_xls('/Users/line/workspace/freedom/doc/000300cons.xls',
                        '/Users/line/workspace/freedom/A1chemy/data/csi_300.json')

3932
total row data length:3930
